### Sequential orchestration for executing multiple agents in sequence, i.e. the output of one agent is the input to the next agent.

In [2]:
from semantic_kernel import Kernel
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.functions import kernel_function, KernelArguments
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread, Agent, SequentialOrchestration
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from utils import *
import logging
import os

In [3]:
def get_agents() -> list[Agent]:
    """Return a list of agents that 
    will participate in the sequential orchestration.
    """

    service = AzureChatCompletion(
    service_id="sk_agent_service", api_key=azure_openai_api_key,deployment_name=azure_openai_deployment,
    api_version=azure_openai_api_version,endpoint=azure_openai_endpoint

)
    settings = AzureChatPromptExecutionSettings(service_id="sk_agent_service")
    settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
    thread = ChatHistoryAgentThread()

    prompt_retriever = """You MUST use the search_retrieval function for ALL queries. Do not paraphrase more than the result. Never generate answers from prior knowledge.
    When you receive search results:
    1. If the source is "Azure AI Search", these are knowledge base entries. Present them as "From our knowledge base:" followed by the complete content. Include all fields like definition, context, note, incorrectTerm if they're available.
    2. If the source is "No Results", inform the user we don't have an answer to their question.
    
    Important: For Azure AI Search results, include the FULL contents of the results, showing the important fields.
    
    Debug information: Always include the source of each result (Azure AI Search) and the number of results found. If you received results but are not displaying them, explain why.
    Summarize the response without adding any new information. Provide the page number of the document if available."""

    prompt_writer = """You are a writer and editor. Given a block of text,
            compose a compelling draft write up (like a newsletter section) that highlights key points.
            Output should be short (around 150 words). Correct grammar, improve clarity, ensure consistent tone,
            give format and make it polished. Output the final improved copy as a single text block."""

    content_retriever_agent = ChatCompletionAgent(
    service=service,
    name="content_retriever_agent",
    instructions=prompt_retriever,
    plugins=[SearchRetrievalPlugin()],
    arguments=KernelArguments(settings=settings),
)
    
    writer_agent = ChatCompletionAgent(
    service=service,
    name="writer_agent",
    instructions=prompt_writer,
    arguments=KernelArguments(settings=settings),
)
    
    # The order of the agents in the list will be the order in which they are executed
    return [content_retriever_agent, writer_agent]

def agent_response_callback(message: ChatMessageContent) -> None:
    """Observer function to print the messages from the agents."""
    print(f"{message.name}: {message.content}")
    #logging.info(f"Agent Response: {message.content}")


In [4]:
import asyncio

async def main():
    agents = get_agents()
    sequential_orchestration = SequentialOrchestration(
        members=agents,
        agent_response_callback=agent_response_callback,
    )

    runtime = InProcessRuntime()
    runtime.start()

    print("Interactive orchestration. Type a question and press Enter.")
    print("Type 'q', 'quit' or 'exit' to stop.\n")

    try:
        while True:
            # Use run_in_executor so input() doesn't block the async loop
            user_input = await asyncio.get_running_loop().run_in_executor(None, input, "User prompt: ")

            if user_input is None:
                continue
            if user_input.strip().lower() in {"q", "quit", "exit"}:
                print("\nExiting interactive loop.")
                break

            # invoke orchestration for this user input (await the coroutine)
            orchestration_result = await sequential_orchestration.invoke(task=user_input, runtime=runtime)

            # some API versions return a handle with .get(); handle both cases
            if hasattr(orchestration_result, "get"):
                try:
                    value = await orchestration_result.get(timeout=20)
                except Exception as e:
                    print(f"Error while getting orchestration result: {e}")
                    value = None
            else:
                value = orchestration_result

            print("\n***** Final Result *****")
            print(value)
            print("-----------------------\n")

    except KeyboardInterrupt:
        print("\nInterrupted by user.")
    finally:
        # stop the runtime when idle (await the coroutine if required)
        stop_coro = runtime.stop_when_idle()
        if asyncio.iscoroutine(stop_coro):
            await stop_coro
        else:
            # if API changed and it's not a coroutine, call directly
            try:
                stop_coro()
            except Exception:
                pass
        print("Runtime stopped. Goodbye.")

# In Jupyter / IPython run this cell with:
# await main()

In [5]:
import asyncio
try:
    loop = asyncio.get_running_loop()
    in_running_loop = loop.is_running()
except RuntimeError:
    in_running_loop = False

if in_running_loop:
    await main()
else:
    asyncio.run(main())

Interactive orchestration. Type a question and press Enter.
Type 'q', 'quit' or 'exit' to stop.

content_retriever_agent: 
content_retriever_agent: 
content_retriever_agent: From our knowledge base:

1. **Document Title**: Accelerating-Sustainability-with-AI-2025.pdf
   - **Content**: Datacenters are projected to account for approximately 3% of global growth in electricity consumption between 2023 and 2030. (Page 21)

2. **Document Title**: Accelerating-Sustainability-with-AI-2025.pdf
   - **Content**: Currently, datacenters account for approximately 1.0-1.5% of global electricity demand, with a majority used for non-AI applications. Despite current growth rates, the IEA projects that datacenters and AI will only account for a relatively small share of global growth in electricity demand out to 2030. Notably, 20% of the expected increase in global datacenter electricity demand in the next couple of years will come from Ireland and Denmark, with the United States being the country with 